###Imports and info

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
file = '/content/drive/MyDrive/The Museum of Edible Earth/EEMasterDF.csv'
df = pd.read_csv(file).reset_index()

file2 = '/content/drive/MyDrive/The Museum of Edible Earth/TNT Categorization.csv'
df2 = pd.read_csv(file2).reset_index()

file3 = '/content/drive/MyDrive/The Museum of Edible Earth/OGTags.csv'
df3 = pd.read_csv(file3).reset_index()

In [ ]:
df.head()

,index,sku,Label name,Country,Country Code,OG_Date,Date,Compositions,Shape,Colour,...,S1,S2,S3,S4,SUM.1,Catalogue,Pure Shrt Description,Full Shrt Description (Upload),Subtitle (Upload),Status
0,0,AM001A,SatinAr Clay,Armenia,AM,9/19/2021,2021-09-19,Clay,Powder,Brown,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,<b>Armenia - AM001A</b><br>Composition: Clay<b...,Armenia - AM001A,Draft
1,1,AM001B,SatinAr Clay,Armenia,AM,2023-08-00,2023-08-01,Clay,Powder,Brown,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Draft
2,2,AT001A,Schindele's Mineralen,Austria,AT,2/2/2022,2022-02-02,Clay,Powder,Brown,...,44.0,NaN,NaN,NaN,44.0,1.0,"<i>Sweet and soft, very fine powder, creamy bu...",<b>Austria - AT001A</b><br>Composition: Clay<b...,Austria - AT001A,Published
3,3,AT002A,Basic Detox Plus,Austria,AT,5/31/2022,2022-05-31,Other,Powder,Grey,...,67.0,72.0,NaN,NaN,139.0,NaN,NaN,<b>Austria - AT002A</b><br>Composition: Other<...,Austria - AT002A,Draft
4,4,AT003A,taste your soil: ars electronica,Austria,AT,9/12/2021,2021-09-12,Sand,Powder,Brown,...,22.0,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,Draft


###Column Name Cleaning

I wanted to make some of the columns and entries tidier and easier to reference, so I changed them here.

In [ ]:
df = df[['sku', 'Label name', 'Country', 'Compositions', 'Shape', 'Colour', 'Date', 'OG_Date', 'How obtained', 'Not eaten']].copy().reset_index()

In [ ]:
df.rename(columns =
          {'sku':'SKU',
           'Label name': 'Name',
           'Compositions':'Composition',
           'Colour':'Color',
           'How obtained':'Acq Method',
           'Date':'Acq Date',
           'Not eaten':'Edibility'},
          inplace=True
         )

In [ ]:
df['Edibility'] = df['Edibility'].replace({'Not eaten':'Not Eaten'})

In [ ]:
df['Composition'] = df['Composition'].replace(
    {'Chak':'Chalk',
     'Termite Mound Earth':'Termite M',
     'Termite Tree Earth':'Termite T'}
)

###Taste and Texture Wrangling

This was likely the most difficult part. A lot of this data was handscraped from the Edible Earth website, so it needed to be organized in a way that could be analyzed. I will document the process step by step.

In df2, there are:
- Descriptions from the website split into singular unique values (Description)
- Encoded tastes (Taste)
- Encoded Textures (Texture)
Ignore the other 3 columns.

In [ ]:
df2.head()

,index,Description,Taste,Texture,taste_can,texture_can,can
0,0,baking soda,"Chemical, Bitter",Powdery,Sweet,Crunchy,Fizzy?
1,1,calcium,Metallic,Chalky,Salty,Grainy,Rough?
2,2,candy,Sweet,Hard,Sour,Powdery,Juicy?
3,3,cinnamon,"Sweet, Spicy",Powdery,Bitter,Smooth,Lumpy?
4,4,dry,NaN,Dry,Spicy,Creamy,NaN


In df3 there are:
- SKU numbers (SKU)
- Descriptions from the website that are grouped with their corresponding SKU (Tags)

In [ ]:
df3.head()

,index,SKU,Tags
0,0,", DE002A","['baking soda', 'calcium', 'candy', 'cinnamon'..."
1,1,", FR002A","['better than seasand', 'calcium', 'candy rock..."
2,2,", US002A","['chalky', 'dry', 'flour', 'hard', 'medicine',..."
3,3,", CI003A","['after rain', 'dusty', 'heavy', 'light', 'min..."
4,4,", DE003A","['chicken food', 'chocolate', 'dry', 'fine', '..."


The goal of this will be to convert the tags (df3, Tags) into encoded values (df2, Taste, Texture).

Here, I stripped the extra tags and text off the raw data, leaving them delimited by a comma and a space for easier splitting, merging and exploding.

In [ ]:
df3['SKU'] = df3['SKU'].str.replace(',', '')
df3['Tags'] = df3['Tags'].str.replace(r'[^a-zA-Z, ]', '', regex = True)
df3.head()

,index,SKU,Tags
0,0,DE002A,"baking soda, calcium, candy, cinnamon, dry, ea..."
1,1,FR002A,"better than seasand, calcium, candy rock, cele..."
2,2,US002A,"chalky, dry, flour, hard, medicine, melts in m..."
3,3,CI003A,"after rain, dusty, heavy, light, mineral, not ..."
4,4,DE003A,"chicken food, chocolate, dry, fine, hagelslag,..."


Here, I mapped converted df2 into a taste and texture map. These maps are dictionaries where the keys are the original descriptors, and the values are the new simplfied values hand-encoded in Excel.

In [ ]:
taste_map = df2.set_index('Description')['Taste'].to_dict()
texture_map = df2.set_index('Description')['Texture'].to_dict()

This is a function created to transform anything in the column of original tags into the new encoded items. This function will:

- Strip and split the items by column
- Apply the taste_map and texture_map values to their matching key value in the column after verifying it's a key value and not null.
- Join the split columns with the same delimiter as before and returns the result as a series.

In [ ]:
def map_taste_texture(entry):
    if pd.isna(entry):
        return pd.Series(["", ""])

    items = [x.strip() for x in entry.split(',')]

    taste_list = [str(taste_map[x]) for x in items if x in taste_map and pd.notna(taste_map[x])]
    texture_list = [str(texture_map[x]) for x in items if x in texture_map and pd.notna(texture_map[x])]

    return pd.Series([', '.join(taste_list), ', '.join(texture_list)])

In [ ]:
df3[['Taste', 'Texture']] = df3['Tags'].apply(map_taste_texture)

These are lambda functions that will ensure that there is only one encoded item per cluster of tags. This means that if one item has multiple tags that correspond to 'sweet', sweet will only appear once in the Taste column.

In [ ]:
df3['Taste'] = df3['Taste'].apply(
    lambda x: ', '.join(dict.fromkeys([i.strip() for i in x.split(',')]))
)

df3['Texture'] = df3['Texture'].apply(
    lambda x: ', '.join(dict.fromkeys([i.strip() for i in x.split(',')]))
)

In [ ]:
df3['SKU'] = df3['SKU'].str.strip()
df['SKU'] = df['SKU'].str.strip()

In [ ]:
df3.head()

,index,SKU,Tags,Taste,Texture
0,0,DE002A,"baking soda, calcium, candy, cinnamon, dry, ea...","Chemical, Bitter, Metallic, Sweet, Spicy, Eart...","Powdery, Chalky, Hard, Dry, Soft, Grainy, Dust..."
1,1,FR002A,"better than seasand, calcium, candy rock, cele...","Tasty, Metallic, Sweet, exp, Rich, Unpleasant,...","Grainy, Chalky, Hard, Crunchy, exp, Dense, Dry..."
2,2,US002A,"chalky, dry, flour, hard, medicine, melts in m...","Bitter, Chemical, Tasty, Salty, exp","Chalky, Dry, Powdery, Hard, Melty, exp"
3,3,CI003A,"after rain, dusty, heavy, light, mineral, not ...","Earthy, Rich, Metallic, Bitter, Sour, Salty, e...","Damp, Dusty, Dense, Airy, Hard, Smooth, Dry, W..."
4,4,DE003A,"chicken food, chocolate, dry, fine, hagelslag,...","Sweet, Chocolatey, Salty, Bitter, exp, Earthy,...","Grainy, Dry, Powdery, Crunchy, Hard, Soft, exp..."


Here, I will join the data by SKU. df comes from the master document, but df2 and df3 come from scraped website data. The SKUs are the common denominator between these datasets.

In [ ]:
merged_df = df.merge(df3[['SKU', 'Tags', 'Taste', 'Texture']], on = 'SKU', how = 'left')

Because this data is delimited by a comma, it must be exploded into individual values in order to be cleaned properly. I will explode them, clean them based on what I found when doing basic exams, and then regroup them.

In [ ]:
df_exploded = df.assign(
Taste = merged_df['Taste'].str.split(',')).explode('Taste')

In [ ]:
df_exploded['Taste'] = df_exploded['Taste'].str.strip()
df_exploded['Taste'] = df_exploded['Taste'].replace({'Swet':'Sweet', 'Earthy. Herbal':'Herbal', 'exp':'Experience'})
df_exploded['Taste'] = df_exploded['Taste'].replace(np.nan, 'N/A')

In [ ]:
dfc_taste = df_exploded.groupby('SKU')['Taste'].apply(lambda x: ', '.join(x)).reset_index()

In [ ]:
df_exploded2 = df.assign(
Texture = merged_df['Texture'].str.split(',')).explode('Texture')

In [ ]:
df_exploded['Texture'] = df_exploded['Texture'].str.strip()

df_exploded['Texture'] = df_exploded['Texture'].replace({
    'Nutty': 'N/A',
    'Milky': 'N/A',
    'Earthy': 'N/A',
    'Umami': 'N/A',
    'Unpalatable': 'Unpleasant',
    'Powder':'Powdery',
    'Dam': 'Damp',
    'Wet':'Damp',
    '':'N/A',
    np.nan: 'N/A',
    'exp':'Experience'
})

In [ ]:
dfc_texture = df_exploded.groupby('SKU')['Texture'].apply(lambda x: ', '.join(x)).reset_index()

The final step is to drop the original columns and merge in the cleaned columns, once again by SKU.

In [ ]:
merged_df.drop(['Taste', 'Texture'], axis = 1, inplace = True)

In [ ]:
merged_df = merged_df.merge(dfc_taste[['SKU','Taste']], on = 'SKU', how = 'left')

In [ ]:
final = merged_df.merge(dfc_texture[['SKU', 'Texture']], on = 'SKU', how = 'left')

In [ ]:
final.head()

,index,SKU,Name,Country,Composition,Shape,Color,Acq Date,OG_Date,Acq Method,Edibility,Tags,Taste,Texture
0,0,AM001A,SatinAr Clay,Armenia,Clay,Powder,Brown,2021-09-19,9/19/2021,Present,Eaten,NaN,N/A,N/A
1,1,AM001B,SatinAr Clay,Armenia,Clay,Powder,Brown,2023-08-01,2023-08-00,Private seller,Eaten,NaN,N/A,N/A
2,2,AT001A,Schindele's Mineralen,Austria,Clay,Powder,Brown,2022-02-02,2/2/2022,Present,Eaten,"calcium, chemical, creamy, fine, iron, mineral...","Metallic, Chemical, Milky, Earthy, Experience,...","Chalky, Creamy, Powdery, Hard, Dry, Experience..."
3,3,AT002A,Basic Detox Plus,Austria,Other,Powder,Grey,2022-05-31,5/31/2022,Shop,Eaten,NaN,N/A,N/A
4,4,AT003A,taste your soil: ars electronica,Austria,Sand,Powder,Brown,2021-09-12,9/12/2021,Project,Not Eaten,NaN,N/A,N/A


And now it's done, the tastes and textures have been cleaned and implemented.

###Date Data Wrangling

In order to properly visualize dates, I will be converting the dates to date time and splitting them by month and year.

In [ ]:
final['Acq Date'] = pd.to_datetime(final['Acq Date'], errors = 'coerce')
final['OG_Date'] = pd.to_datetime(final['OG_Date'], errors = 'coerce')
final['Acq Year'] = final['Acq Date'].dt.year
final['Acq Month'] = final['OG_Date'].dt.month

Also for some reason the date 1900 wound up in the dataset. I didn't notice till I started the analysis, so I had to go back and pluck it out here.

In [ ]:
final = final[final['Acq Year'] != 1900]

In [ ]:
final['Acq Month'] = final['Acq Month'].astype(float)
final['Acq Month'] = pd.to_datetime(final['Acq Month'], format='%m')
final['Acq Month'] = final['Acq Month'].dt.strftime('%B')

###Final Check and Download

In [ ]:
final.head()

,index,SKU,Name,Country,Composition,Shape,Color,Acq Date,OG_Date,Acq Method,Edibility,Tags,Taste,Texture,Acq Year,Acq Month
0,0,AM001A,SatinAr Clay,Armenia,Clay,Powder,Brown,2021-09-19,2021-09-19,Present,Eaten,NaN,N/A,N/A,2021.0,September
1,1,AM001B,SatinAr Clay,Armenia,Clay,Powder,Brown,2023-08-01,NaT,Private seller,Eaten,NaN,N/A,N/A,2023.0,NaN
2,2,AT001A,Schindele's Mineralen,Austria,Clay,Powder,Brown,2022-02-02,2022-02-02,Present,Eaten,"calcium, chemical, creamy, fine, iron, mineral...","Metallic, Chemical, Milky, Earthy, Experience,...","Chalky, Creamy, Powdery, Hard, Dry, Experience...",2022.0,February
3,3,AT002A,Basic Detox Plus,Austria,Other,Powder,Grey,2022-05-31,2022-05-31,Shop,Eaten,NaN,N/A,N/A,2022.0,May
4,4,AT003A,taste your soil: ars electronica,Austria,Sand,Powder,Brown,2021-09-12,2021-09-12,Project,Not Eaten,NaN,N/A,N/A,2021.0,September


In [ ]:
final.columns

Index(['index', 'SKU', 'Name', 'Country', 'Composition', 'Shape', 'Color',
       'Acq Date', 'OG_Date', 'Acq Method', 'Edibility', 'Tags', 'Taste',
       'Texture', 'Acq Year', 'Acq Month'],
      dtype='object')

In [ ]:
final.set_index('SKU', inplace = True)

Here is the final clean file, ready for analysis.

In [ ]:
final.to_csv('clean_dirt_final.csv', index = False, encoding='utf-8-sig')

Here, I will download the exploded variants of the dataset as well. This is necessary, because I will be using the data in Tableau, and need the tastes and textures expanded to do so.

In [ ]:
df_exploded.to_csv('taste_final.csv', index = False, encoding='utf-8-sig')
df_exploded2.to_csv('texture_final.csv', index = False, encoding='utf-8-sig')